https://kormilitzin.medium.com/med7-clinical-information-extraction-system-in-python-and-spacy-5e6f68ab1c68

# **INSTALLING LLIBRARIES AND DEPENDENCIES** 

In [7]:
#!sudo apt install tesseract-ocr
!pip install Image
!pip install 'camelot-py[cv]'
!pip install 'camelot-py[all]'
!pip install pytesseract
!pip install nltk
!pip install stopWords
!pip install pandas
#!pip install --no-cache-dir https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl
!pip install tabula-py

In [9]:
!pip install pydub
!pip install nameparser
!pip install pysummarization

**LIBRARIES FOR IMAGE TO TEXT CONVERSION**

In [26]:
!pip install pdf2image
!brew install poppler-qt5

To reinstall 24.11.0, run:
  brew reinstall poppler-qt5


*Camelot is a Python library that can help you extract tables from PDFs!*

In [28]:
import camelot.io as camelot
import re

ImportError: PyO3 modules compiled for CPython 3.8 or older may only be initialized once per interpreter process

In [16]:
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [ ]:
# from google.colab import files
# uploaded = files.upload()

*Extracting the text from image 'image1.jpg' using pytesseract library and then printing it*

In [18]:
extractedInformation = pytesseract.image_to_string(Image.open('image1.jpg'))

In [20]:
print(extractedInformation)

=
f I Medicity | ¢
ae: | S|@©|
U.N. Mehta Institute of Cardiology & Research Centre
(Affiliated to BJ. Medical College, Ahmedabad)
Civil Hospital Campus (Medicity), Asarwa, Ahmedabad-380016. Gujarat.
Phone : (079) 22682395, 22684220, 22684200 Fax : 079-22682092
Email : unmicrc@gmail.com Website : www-unmicrc.org

Coronary Angioplasty

Patient :Lalchand Sughomal Lachhani (68y 6m / Male) Patient ID : UNM-2015-08-083 152
IPD No. : IPD/2022/02/0005003 Admission Date : 20/02/2022
Date of Procedure : 25/02/2022 Angio No. : A-2118

Time of Procedure : 17:08

Primary Operator : Dr. Hasit Joshi (G-19508) Professor in Cardiology
Secondary Operator : Dr. Riyaz Didarali Charaniya (G-29637) Assistant Professor in Cardiology

Indication

- Other Indications : OLD ACS,CAD, S/p PTCA to RCA(2018), P/w ACS-NSTEMI, CAD-TVD
- PAMI: No

- ACS: No

Procedure & Description

- Status Of Procedure : Elective -

- Procedure Approach : Radial Right

- Support Done : IABP Status : Unused , Procedure Time : 55 min

# FORMATTING THE TEXT FROM IMAGE

*NLTK (Natural Language Toolkit) Library is a suite that contains libraries and programs for statistical language processing. It is one of the most powerful NLP libraries, which contains packages to make machines understand human language and reply to it with an appropriate response.*

In [22]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
 
stopWords = set(stopwords.words('english'))
sent = sent_tokenize(extractedInformation)
wordsFiltered=[]
for w in sent:
  wordsFiltered.append(w)
word=[]
patterns = "Drug"
for w in wordsFiltered:
  x= word_tokenize(w)
  for i in x:
    if re.match(patterns,i):
      print(x)

['Stent', 'Name', ':', 'XIENCE', 'PRIME', 'LL', 'Stent', 'Type', ':', 'Drug', 'Elluting', 'Stent', '-', 'Everolimus', 'Diameter', ':', '2.5', 'Lenght', ':', '38', 'Time', ':', '10', 'Max', 'Pressure', ':', '12', 'atm', 'Postdialatation', 'Balloon', '|1', '.']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/virajpatel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/virajpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CONVERTING TO DATAFRAME

In [24]:
import camelot 
import pandas as pd
tables = camelot.read_pdf('Coronary Angioplasty.pdf')

#tables[0].to_csv('camelot.csv')
#print(len(tables))
tables[0].df

ImportError: PyO3 modules compiled for CPython 3.8 or older may only be initialized once per interpreter process

*spaCy  is a free, open-source library for advanced Natural Language Processing (NLP) in Python.**spaCy** is designed specifically for production use and helps you build applications that process and “understand” large volumes of text. It can be used to build information extraction or natural language understanding systems, or to pre-process text for deep learning.*

In [ ]:
import spacy

med7 = spacy.load("en_core_med7_lg")

# create distinct colours for labels
col_dict = {}
seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
doc = med7(text)

spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

[(ent.text, ent.label_) for ent in doc.ents]

[('Magnesium hydroxide', 'DRUG'),
 ('suspension', 'FORM'),
 ('PO', 'ROUTE'),
 ('30ml', 'DOSAGE'),
 ('bid', 'FREQUENCY'),
 ('for the', 'DURATION')]

In [ ]:
import tabula
dataframe = tabula.read_pdf('Coronary Angioplasty.pdf',pages="all")
dataframe

# EXTRACTING INFORMATION FROM TEXT FILE

**'find_next' function uses regex expressions to search and extract different generalized information from the extracted text file**

In [ ]:
def find_next(words,data):
  text=""
  for word in words:
    if(words[0].lower()=='age'):
     text=re.search('\d+(?=[\s]?y | year)',data.lower())
    if(text):
      break
    text=re.search('(?<='+word.lower()+' : )(\S*)',data.lower())
    if(text):
      break
  return text

In [ ]:
from pdf2image import convert_from_path
def extract_pdf2img(path):
  images = convert_from_path(path)
  for i in range(len(images)):
    images[i].save('page'+ str(i) +'.jpg', 'JPEG')
  return images


In [ ]:
images=extract_pdf2img('/content/Coronary Angioplasty.pdf')

In [ ]:
all_text_data=[]
file = open('report.txt','w')
for i in range(len(images)):
  extractedInformation = pytesseract.image_to_string(Image.open('/content/page'+str(i)+'.jpg'))
  sent = sent_tokenize(extractedInformation)
  wordsFiltered=[]
  for w in sent:
    file.write(w)
# all_text_data\
file.close()

In [ ]:

# def find_next(words,data):
  # for word in words:
data='''
Coronary Angiography

Patient :Lalchand Sughomal Lachhani (68y 6m/ Male) _ Patient ID : UNM-2015-08-083152
IPD No. : IPD/2022/02/0005003 Admission Date : 20/02/2022

Date of Procedure : 23/02/2022 Angio No. : D-42737

Time of Procedure : 08:23

Primary Operator : Dr. Hasit Joshi (G-19508) Professor in Cardiology
Secondary Operator : Dr. Riyaz Didarali Charaniya (G-29637) Assistant Professor in Cardiology

 

Indication

- Ischemic Heart Disease Type : CSA

- PAMI: No

- ACS: No

Procedure & Description

- Procedure Type : Coronary Angiography

- Status of Procedure : Elective , Procedure Approach : Radial - Left

- Diagnostic Catheter : Left Coronary TIGER 5F ,Right Coronary TIGER 5F ,Guide wire Non-hydrophilic 03 5,Contrast
Agent Nonionic Low-osmolar,Contrast Volume 100 ml

- Anesthesia used : LOCAL
- Total Radiation Dose at Source : 219.0 mGy

Peripheral Stenosis

Coronary Anatomy

- Extent of Disease : Triple Vessel, Dominance : Right
LMCA | LMCA,Detail : Normal

Mid LAD, Stenosis Severity : 90 %,

Proximal LAD,Detail : Plaque

Distal Circumflex,Stenosis Severity : 80 %,
RCA | Distal RCA,Lesion's Remarks: IN STENT- 70%

 

LAD

 

 

 

Final Diagnosis
- OLD ACS, HTN, CAD-TVD, WITH RIGHT RENAL ARTERY, S/P PTCA TO RIGHT PTRA (27/11/2011), S/P
PCI TO RCA (INTEGRITY 27/11/2015)(2.75x18), P/W ACS NSTEMI,
- LV Function : Severe LVD
- TVD : Yes

- Remarks : RIGHT RENAL - 50% ISR IN STENT, LEFT RENAL- 60% STENOSIS

Advice :
PTCA TO LAD & RCA

Dr. Hasit Joshi (G-19508) Professor in Cardiology :

 

Dr. Riyaz Didarali Charaniya (G-29637) Assistant Professor in Cardiology :

 

Generated By : Jayshree S. Sanchaniya

unmicrc/MRD/Rep-A/02/rev-02 1-Aug-17

 

Patient :Lalchand Sughomal Lachhani (68y 6m / Male) Patient ID : UNM-2015-08-083 152
IPD No: IPD/2022/02/0005003 Admission Date : 20/02/2022

Date of Procedure : 23/02/2022 Angio No. : D-42737

Time of Procedure : 08:23

 

unmicrc/MRD/Rep-A/02/rev-02 1-Aug-17
'''
# 

In [ ]:
file = open('report.txt', 'r')
# x=""
# for each in file:
#     print(each)
#     words=['Procedure','Time of Procedure']
#     x=find_next(words,each)
# print(x)

**final info is a dictionary in which every information is stored with its corresponding key**

In [ ]:
final_info={}

**add_result function has three different arguments**


*   words- list of keywords to find.
*   key- the name of key by which the information is stored.
*   data- Whole text file in which keywords are searched and extracted





In [ ]:
def add_result(words,key,data):
  text=find_next(words,data)
  if(text):
    final_info[key]=text.group(0)
    return

## Adding every general paramater

In [ ]:
#Age
add_result(['Age'],"Age",data)

In [ ]:
#Duration of Procedure
add_result(['Time of Procedure','Duration of Procedure'],'Duration of Procedure',data)
final_info

In [ ]:
#Procedure
add_result(['Procedure Type'],'Procedure Type',data)
final_info

In [ ]:
#Status Of Procedure
add_result(['Status Of Procedure'],'Status Of Procedure',data)
final_info

In [ ]:
#Procedure Approach
add_result(['Procedure Approach'],'Procedure Approach',data)
final_info

In [ ]:
#Ischemic Heart Disease Type
add_result(['Ischemic Heart Disease Type'],'Ischemic Heart Disease Type',data)
final_info

In [ ]:
#advise
word='advise'
x=re.search('(?<='+word.lower()+')'+'[\r\n]+([^\r\n]+)',data.lower())
x

In [ ]:
#comments
word='Comments'
x=re.search('(?<='+word.lower()+')'+'[\r\n]+([^\r\n]+)',data.lower())
x

In [ ]:
def find_next2(words,data,key):
    for word in words:
        text=re.search('(?<='+word.lower()+': )(\S*)',data.lower())
        if(text):
            final_info[key]=text.group(0)

In [ ]:
find_next2("ACS",data,"ACS")

In [ ]:
find_next2("PAMI",data,"PAMI")

In [ ]:
final_info

In [ ]:
def find_remarks(words,key,data):
    for word in words:
        text=re.search('(?<='+word.lower()+' :'+')'+'([^\r\n]+)',data.lower())
        print(text)
        if(text):
            final_info[key]=text.group(0)

find_remarks(['remarks','comments'],'Remarks',data)
final_info

In [ ]:
def find_advice(words,key,data):
    for word in words:
        text=re.search('(?<=Advice :)([\r\n])([^\r\n]+)',data)
        text1=re.search('(?<=Advice)([\r\n])([^\r\n]+)',data)
        #print(text)
        if(text):
            final_info[key]=text.group(0)
        if(text1):
            final_info[key]=text1.group(0)
find_advice(['advice','advise'],'Advice',data)
final_info

In [ ]:
def name(word,key):
  file=open('/content/report.txt','r')
  Lines = file.readlines()
  
  count = 0
  # Strips the newline character
  for line in Lines:
      # line=line.lower()
      if word in line:
          # word=word.lower()
          x=re.findall(r"[A-Z][a-z]+,?\s+(?:[A-Z][a-z]*\.?\s*)?[A-Z][a-z]+", line)
          for j in x:
            final_info[key]=j

In [ ]:
name("Patient","Patient")
name("Primary Operator","Doctor")

In [ ]:
final_info

{'ACS': 'no',
 'Advice': '\nPTCA TO LAD & RCA',
 'Age': '68',
 'Doctor': 'Hasit Joshi',
 'Duration of Procedure': '08:23',
 'Ischemic Heart Disease Type': 'csa',
 'PAMI': 'no',
 'Patient': 'Lalchand Sughomal Lachhani',
 'Procedure Approach': 'radial',
 'Procedure Type': 'coronary',
 'Remarks': ' right renal - 50% isr in stent, left renal- 60% stenosis',
 'Status Of Procedure': 'elective'}

In [ ]:
import spacy

med7 = spacy.load("en_core_med7_lg")

# create distinct colours for labels
col_dict = {}
seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

file=open('/content/report.txt','r')
Lines = file.readlines()

for line in Lines:
  doc = med7(line)
  for ent in doc.ents:
    if(ent.label_.lower()=="drug"):
      final_info['Drug']=ent.text
  # print([(ent.text, ent.label_) for ent in doc.ents])
  # spacy.displacy.render(doc, style='ent', jupyter=True, options=options)
  # [(ent.text, ent.label_) for ent in doc.ents]
print(final_info)

{'Age': '68', 'Duration of Procedure': '08:23', 'Procedure Type': 'coronary', 'Status Of Procedure': 'elective', 'Procedure Approach': 'radial', 'Ischemic Heart Disease Type': 'csa', 'ACS': 'no', 'PAMI': 'no', 'Remarks': ' right renal - 50% isr in stent, left renal- 60% stenosis', 'Advice': '\nPTCA TO LAD & RCA', 'Patient': 'Lalchand Sughomal Lachhani', 'Doctor': 'Hasit Joshi', 'Drug': 'unmicrc'}


In [ ]:
import pandas as pd
df=pd.DataFrame(data=final_info,index=[0])
print (df)
df.to_excel('dict1.xlsx')

  Age Duration of Procedure Procedure Type Status Of Procedure  \
0  68                 08:23       coronary            elective   

  Procedure Approach Ischemic Heart Disease Type ACS PAMI  \
0             radial                         csa  no   no   

                                             Remarks               Advice  \
0   right renal - 50% isr in stent, left renal- 6...  \nPTCA TO LAD & RCA   

                      Patient       Doctor     Drug  
0  Lalchand Sughomal Lachhani  Hasit Joshi  unmicrc  


In [ ]:
# Comments
# - Type Of Procedure : Multi Vessel Angioplasty
# - Lesion Summary : Lesions Attempted-1 , Lesions Stented-1 , Stent Used-1 , Lesions Successfully Dilated-1
# Final Diagnosis
# - Procedure Result : Successful PTCA to LAD done
# Advise
# -Regular Follow-up & Medicine : Staged PTCA to RCA

# EXTRACTING FROM SPEECH/AUDIO FILES

In [ ]:
#Speech to text 
#import libraries

import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile('/content/VOXTAB_Medical_audio.wav') as source:
    audio_text = r.listen(source)
    try:
        text = r.recognize_google(audio_text)
        print('Converting Audio to TEXT....')
        print(text)
    except:
         print('Run again, API Error !')

Converting Audio to TEXT....
the patient is a 49 year old white female establish Vista Dermatology lasting office on the 8th of 10th 2004 she comes in today for revitalization of acne plus she has it was a rush for the past two months now and I trashed stomach neck and back on examination


In [ ]:
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence


r = sr.Recognizer()

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    sound = AudioSegment.from_wav(path)  
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS-14,

        keep_silence=500,
    )
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    return whole_text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
text=get_large_audio_transcription('/content/JonesMedical.wav')

audio-chunks/chunk1.wav : Hi this is doctor helen white. 
audio-chunks/chunk2.wav : Today's the 16th of june. 
audio-chunks/chunk3.wav : I'm doing a referral after please. 
audio-chunks/chunk4.wav : All reference h.w. / ca. 
audio-chunks/chunk5.wav : He could date today's date. 
audio-chunks/chunk6.wav : Pocket private confidential full mister jason. 
audio-chunks/chunk7.wav : Spring consultant orthopedic surgeon. 
audio-chunks/chunk8.wav : Whiskey hospital davis teespring. 
audio-chunks/chunk9.wav : Ari. 
audio-chunks/chunk10.wav : Thomas jones date of birth. 
audio-chunks/chunk11.wav : 22nd of june 1972 address one the house to the road. 
audio-chunks/chunk12.wav : Liverpool l13 pa. 
audio-chunks/chunk13.wav : Telephone number. 
audio-chunks/chunk14.wav : Oh 77421 932 18. 
audio-chunks/chunk15.wav : So you could see this 38 year old gentleman. 
audio-chunks/chunk16.wav : Mr. jones is a nice young gentleman who tell it be since buying tight footwear has suffered from left medial foot 

In [ ]:
text

"Hi this is doctor helen white. Today's the 16th of june. I'm doing a referral after please. All reference h.w. / ca. He could date today's date. Pocket private confidential full mister jason. Spring consultant orthopedic surgeon. Whiskey hospital davis teespring. Ari. Thomas jones date of birth. 22nd of june 1972 address one the house to the road. Liverpool l13 pa. Telephone number. Oh 77421 932 18. So you could see this 38 year old gentleman. Mr. jones is a nice young gentleman who tell it be since buying tight footwear has suffered from left medial foot arch pain. Zane is infiltration system. I need some food from tuesday or thursday. . he finds it the pain gets worse after he receives his exercise regime.. All examinations new baby tendinous he's somewhat 10 to over the midpoint of his middle media arts. Play left foot. you parrot. I want your sister's unresolved spotify. What he would benefit from steroid injection. I would be grateful for your expert opinion thank you very much s

In [ ]:
# inp = sr.AudioFile('/content/JonesMedical.wav')
# with inp as file:
#   audio = r.record(file)
#   print(r.recognize_google(audio))
#   # print(audio)

hi this is Doctor Helen White today's the 16th of June I'm doing a referral lots of please all reference HW / CA you could dated today's date private confidential for mr. Jason spring consultant Orthopaedic surgeon whiskey Hospital Thomas Jones


In [ ]:
from nameparser import HumanName
# for line in text:
#   print(line)
text_sent=sent_tokenize(text)
# name=""
for sent in text_sent:
  name=HumanName(sent)
  print(name)

Hi this is doctor helen white.
Today's the 16th of june.
I'm doing a referral after please.
All reference h.w.
/ ca.
He could date today's date.
Pocket private confidential full mister jason.
Spring consultant orthopedic surgeon.
Whiskey hospital davis teespring.
Ari.
Thomas jones date of birth.
22nd of june 1972 address one the house to the road.
Liverpool l13 pa. Telephone number.
Oh 77421 932 18.
So you could see this 38 year old gentleman.
Mr. jones is a nice young gentleman who tell it be since buying tight footwear has suffered from left medial foot arch pain.
Zane is infiltration system.
I need some food from tuesday or thursday.
.
he finds it the pain gets worse after he receives his exercise regime.. All examinations new baby tendinous he's somewhat 10 to over the midpoint of his middle media arts.
Play left foot.
you parrot.
I want your sister's unresolved spotify.
What he would benefit from steroid injection.
I would be grateful for your expert opinion thank you very much st

In [ ]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor
auto_abstractor = AutoAbstractor()
auto_abstractor.tokenizable_doc = SimpleTokenizer()
auto_abstractor.delimiter_list = [".", "\n"]
abstractable_doc = TopNRankAbstractor()
result_dict = auto_abstractor.summarize(text, abstractable_doc)
for sentence in result_dict["summarize_result"]:
    print(sentence)

Hi this is doctor helen white.

 Today's the 16th of june.

 I'm doing a referral after please.

 He could date today's date.

 Pocket private confidential full mister jason.

 Thomas jones date of birth.

 22nd of june 1972 address one the house to the road.

 So you could see this 38 year old gentleman.

 jones is a nice young gentleman who tell it be since buying tight footwear has suffered from left medial foot arch pain.

 I need some food from tuesday or thursday.

